In [8]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

!pip uninstall -y langchain langchain-core langchain-community langchain-text-splitters langchain_groq > /dev/null 2>&1
!pip install -q requests==2.32.4
!pip install -q langchain langchain-community langchain-text-splitters langchain_groq faiss-cpu sentence-transformers pypdf

from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader("/content/mental_health_Document.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(documents)

print("Loading embeddings and creating vector store...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.4)

template = """Answer the question based only on the following context:

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("\n" + "="*50)
print("RAG System Ready! Ask questions about the document.")
print("Type 'quit' or 'exit' to stop.")
print("="*50 + "\n")

while True:
    query = input("Your Question: ").strip()

    if query.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break

    if not query:
        print("Please enter a question.\n")
        continue

    try:
        result = rag_chain.invoke(query)
        print(f"\nAnswer: {result}\n")
        print("-"*50 + "\n")
    except Exception as e:
        print(f"Error: {e}\n")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, which is not installed.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, which is not installed.
google-cloud-bigquery 3.38.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-cloud-bigquery 3.38.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
Loading embeddings and creating vector store...

RAG System Ready! Ask questions about the document.
Type 'quit' or 'exit' to stop.

Your Question: what